In [1]:
import os
import sys
import math
import pandas as pd
import nltk as nl
import numpy as np
import json
import gensim.models.keyedvectors as word2vec
import re
import skfuzzy as fuzz
from jgtextrank import keywords_extraction

import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

In [2]:
# from nltk.stem.porter import *
from functools import singledispatch
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from string import punctuation

from collections import Counter

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from pyeasyga import pyeasyga
# from geneticalgorithm import geneticalgorithm as ga

from lexrank import STOPWORDS, LexRank

2021-04-26 22:17:32,050 - filelock - INFO - Lock 1714673706224 acquired on C:\ProgramData\Anaconda3\lib\site-packages\urlextract\data\tlds-alpha-by-domain.txt.lock
2021-04-26 22:17:32,078 - filelock - INFO - Lock 1714673706224 released on C:\ProgramData\Anaconda3\lib\site-packages\urlextract\data\tlds-alpha-by-domain.txt.lock


In [3]:
# from pprint import pprint
from fuzzy_logic.terms import Term
from fuzzy_logic.variables import FuzzyVariable
from fuzzy_logic.mamdani_fs import MamdaniFuzzySystem
from fuzzy_logic.mf import TriangularMF

In [4]:
@singledispatch
def to_serializable(val):
    """Used by default."""
    return str(val)

@to_serializable.register(np.float32)
def ts_float32(val):
    """Used if *val* is an instance of numpy.float32."""
    return np.float64(val)

# Text Pre-Processing

In [5]:
# stemmer = PorterStemmer()
stemmer = SnowballStemmer(language='english')

In [6]:
def remove_stop_words(words):
    """
    Will remove stop words from a list
    :param words: input list
    :return: output list of words without stop words.
    """
    custom_stop_words = set(stopwords.words('english') + list(punctuation))
    return [stemmer.stem(word) for word in words if word not in custom_stop_words]

In [7]:
def tokenize(text, stop_words):
    """
    Tokenizes a given text and also removes stop words.
    :param text:
    :param stop_words:
    :return:
    """
    words = nl.word_tokenize(text)
    words = [w.lower() for w in words]
    return [w for w in words if w not in stop_words and not w.isdigit()]

In [8]:
def text_cleaner(in_text):
    """

    :param in_text:
    :return:
    """
    cleaned_text = re.sub(r'([a-zA-Z])\\1{2,}', r'$1', in_text)
    cleaned_text = re.sub("\S*\d\S*", "", cleaned_text).strip()
    return re.sub(r'[^a-zA-Z0-9\s,.]', '', cleaned_text)

In [9]:
def get_cleaned_text(text):
    """

    :param text:
    :return:
    """

    in_text = text.lower()
    cleaned_text = text_cleaner(in_text)
    tokens = nl.word_tokenize(cleaned_text)

    # TODO : Use setemimng here.
    return remove_stop_words(tokens)

# Util Functions

In [10]:
def get_sen_length(sen):
    return len(nl.word_tokenize(sen))

In [11]:
# def get_sen_pos(postion, total_sens):
#     postion = 1 if postion == 0 else postion
#     return (postion - 1) / total_sens

def get_sen_pos(postion, total_sens):
    postion += 1
    postion = 1 if postion == total_sens else postion
    postion = 2 if postion == (total_sens - 1) else postion
    return (total_sens - postion) / total_sens

In [12]:
def get_proper_nouns(sentence):
    nnp = [word for word, pos in nl.pos_tag(nl.word_tokenize(str(sentence))) if pos == 'NNP']
    return len(nnp)

In [13]:
def extract_np(psent):
    for subtree in psent.subtrees():
        if subtree.label() == 'NP':
            yield ' '.join(word for word, tag in subtree.leaves())

In [14]:
def get_np_vps(sen):
    # Todo check grammar for verb phrases
    grammar = r"""
          NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
          {<NNP>+}                # chunk sequences of proper nouns
          {<NN>+}                 # chunk consecutive nouns
          VP: {<VB.*><NP|PP|CLAUSE>+$}
          """
    cp = nl.RegexpParser(grammar)

    tagged_sent = nl.pos_tag(sen.split())
    parsed_sent = cp.parse(tagged_sent)
    nps = []
    for npstr in extract_np(parsed_sent):
        nps.append(npstr)
    return nps

In [15]:
def vectorize_sent(sent, model):
    # print(pre_process(sent))
    return np.mean([model[w] for w in get_cleaned_text(sent) if w in model]
                   or [np.zeros(300)], axis=0)

In [16]:
def get_sen_vec_list(sens, model):
    index_vec_list = []
    for i, sen in enumerate(sens):
        vec = vectorize_sent(sen, model)
        index_vec_list.append(vec)
    return index_vec_list

In [17]:
def calculate_tf_idf_sum(sen, D, df_map):
    tokens = get_cleaned_text(sen)
    tf_map = dict(Counter(tokens))
    tf_idf_map = {k: (v * math.log(D / df_map.get(k, 1))) for k, v in tf_map.items()}
    return sum(tf_idf_map.values())

In [18]:
def get_sen_cohesiveness(sens, model):
    index_vec_list = get_sen_vec_list(sens, model)
    sen_score_map = {}
    # print(index_vec_list)
    c_s_array = (cosine_similarity(index_vec_list, index_vec_list))
    cohesiveness = (c_s_array.sum(axis=0)) - 1
    return {i: v for i, v in enumerate(cohesiveness)}

In [19]:
def get_sentences(text):
    return nl.sent_tokenize(text)

In [20]:
def get_val(val, min_val, max_val):
    min_val = 1 if min_val <= 0 else min_val
    return (val - min_val) / (max_val - min_val)

In [21]:
def load_model(model_path):
    return word2vec.KeyedVectors.load_word2vec_format(model_path, binary=True)

# Base Summrizer

In [22]:
# nltk.download('reuters')
from nltk.corpus import reuters, stopwords

In [23]:
def normalize_values(res_arr):
    # print([x['tf_idf'] for x in res_arr])
    min_max_map = {}
    for k in ['tf_idf', 'sen_length', 'proper_nouns', 'np_vps', 'c_score', 'sen_pos']:
        min_max_map.setdefault(k, {})['max'] = max([x[k] for x in res_arr])
        min_max_map[k]['min'] = min([x[k] for x in res_arr])
    result = []
    # print(min_max_map)
    for r in res_arr:
        tf_idf = get_val(r['tf_idf'], min_max_map['tf_idf']['min'], min_max_map['tf_idf']['max'])
        sen_length = get_val(r['sen_length'], min_max_map['sen_length']['min'], min_max_map['sen_length']['max'])
        proper_nouns = get_val(r['proper_nouns'], min_max_map['proper_nouns']['min'],
                               min_max_map['proper_nouns']['max'])
        np_vps = get_val(r['np_vps'], min_max_map['np_vps']['min'], min_max_map['np_vps']['max'])
        c_score = get_val(r['c_score'], min_max_map['c_score']['min'], min_max_map['c_score']['max'])
        sen_pos = r['sen_pos']
        total_score = 0.7 * tf_idf + 0.05 * sen_length + 0.025 * proper_nouns + \
                      0.025 * np_vps + 0.15 * c_score + 0.15 * sen_pos
        obj = {'tf_idf': 0.7 * tf_idf, 'sen_length': 0.05 * sen_length, 'proper_nouns': 0.025 * proper_nouns,
               'np_vps': 0.025 * np_vps,
               'c_score': 0.15 * c_score, 'sen_pos': 0.15 * sen_pos, 'total_score': total_score, 'index': r['index']}
        result.append(obj)
    return result

In [24]:
def get_final_sen_ranks(text, D, df_map, model):
    curpath = os.path.abspath(os.curdir)
    sens = get_sentences(text)
    t = len(sens)
    coh_map = get_sen_cohesiveness(sens, model)
    result_array = []

    for i, s in enumerate(sens):
        # print(s)
        tf_idf = calculate_tf_idf_sum(s, D, df_map)
        sen_length = get_sen_length(s)
        proper_nouns = get_proper_nouns(s)
        np_vps = len(get_np_vps(s))
        c_score = coh_map[i]
        pos = get_sen_pos(i, t)
        rank_obj = {'tf_idf': tf_idf, 'sen_length': sen_length, 'proper_nouns': proper_nouns,
                    'np_vps': np_vps, 'c_score': c_score, 'sen_pos': pos, 'index': i}
        # print("   ", rank_obj)
        result_array.append(rank_obj)
    result = normalize_values(result_array)
    wr = open(os.path.join(curpath, '/tmp/score.json'), "w")
    for i, r in enumerate(result):
        r["sentence"] = sens[i]
        wr.write(json.dumps(r) + "\n")
    return result, sens

In [25]:
def return_postive_value(val):
    if(val < 0):
        val = -val
    return val

In [26]:
def fuzzy_model(res_array):
    min_max_map = {}    
    result = []
    
    t1 = Term('L', TriangularMF(0, 0.1, 0.2))
    t2 = Term('H', TriangularMF(0.2, 0.5, 1.0))

    tf_idf: FuzzyVariable = FuzzyVariable('tf_idf', 0.0, 1.0, t1, t2)
    sen_length: FuzzyVariable = FuzzyVariable('sen_length', 0.0, 1.0, t1, t2)
    proper_nouns: FuzzyVariable = FuzzyVariable('proper_nouns', 0.0, 1.0, t1, t2)
    np_vps: FuzzyVariable = FuzzyVariable('np_vps', 0.0, 1.0, t1, t2)
    c_score: FuzzyVariable = FuzzyVariable('c_score', 0.0, 1.0, t1, t2)
    sen_pos: FuzzyVariable = FuzzyVariable('sen_pos', 0.0, 1.0, t1, t2)
        
    output = FuzzyVariable(
        'output', 0.0, 1.0,
        Term('unimportant', TriangularMF(0.0, 0.1, 0.2)),
        Term('average', TriangularMF(0.2, 0.3, 0.4)),
        Term('good', TriangularMF(0.4, 0.5, 0.7)),
        Term('important', TriangularMF(0.7, 0.8, 1)),
    )
    
    mf: MamdaniFuzzySystem = MamdaniFuzzySystem([tf_idf, sen_length, proper_nouns, np_vps, c_score, sen_pos], [output])

    fuzzy_rules = pd.read_csv(r'C:\Users\Mohsin\Desktop\Desktop\Fuzzy Logics Rules (Thesis)\rules_v3.csv')
    for item in fuzzy_rules.values:
        mf.rules.append(mf.parse_rule("if (tf_idf is " + item [0] + ") and (sen_length is " + item [1] + ") and (proper_nouns is " + item [2] + ") and (np_vps is " + item [3] + ") and (c_score is " + item [4] + ") and (sen_pos is " + item [5] + ") then (output is " + item [6] + ")"))
        
        
    for k in ['tf_idf', 'sen_length', 'proper_nouns', 'np_vps', 'c_score', 'sen_pos']:
        min_max_map.setdefault(k, {})['max'] = max([x[k] for x in res_array])
        min_max_map[k]['min'] = min([x[k] for x in res_array])

        
    for r in res_array:
        
        tfIdf = get_val(r['tf_idf'], min_max_map['tf_idf']['min'], min_max_map['tf_idf']['max'])
        senLength = get_val(r['sen_length'], min_max_map['sen_length']['min'], min_max_map['sen_length']['max'])
        properNouns = get_val(r['proper_nouns'], min_max_map['proper_nouns']['min'], min_max_map['proper_nouns']['max']) 
        npVps = get_val(r['np_vps'], min_max_map['np_vps']['min'], min_max_map['np_vps']['max'])
        cScore = get_val(r['c_score'], min_max_map['c_score']['min'], min_max_map['c_score']['max']) 
        senPos = ((r['sen_pos'] - min_max_map['sen_pos']['min'])/(min_max_map['sen_pos']['max'] - min_max_map['sen_pos']['min']))
        
        resultant_value = mf.calculate({
            tf_idf: ts_float32(return_postive_value(tfIdf)), 
            sen_length: ts_float32(return_postive_value(senLength)), 
            proper_nouns: ts_float32(return_postive_value(properNouns)), 
            np_vps: ts_float32(return_postive_value(npVps)), 
            c_score: ts_float32(return_postive_value(cScore)), 
            sen_pos: ts_float32(return_postive_value(senPos))
        })
        
        fuzzy_score = [val for val in resultant_value.values()]
        
        print("fuzzay Score: ", fuzzy_score)
        
        obj = {
            'tf_idf': ts_float32(return_postive_value(tfIdf)),
            'sen_length': ts_float32(return_postive_value(senLength)),
            'proper_nouns': ts_float32(return_postive_value(properNouns)),
            'np_vps': ts_float32(return_postive_value(npVps)),
            'c_score': ts_float32(return_postive_value(cScore)),
            'sen_pos': ts_float32(return_postive_value(senPos)),
            'total_score': fuzzy_score[0],
            'index': r['index']
        }
        
        result.append(obj)
    
    return result

In [27]:
def fuzzy_ranking_model(text, D, df_map, model):
    curpath = os.path.abspath(os.curdir)
    sens = get_sentences(text)
    t = len(sens)
    coh_map = get_sen_cohesiveness(sens, model)
    result_array = []
    
    
    for i, s in enumerate(sens):
        # print(s)
        tf_idf = calculate_tf_idf_sum(s, D, df_map)
        sen_length = get_sen_length(s)
        proper_nouns = get_proper_nouns(s)
        np_vps = len(get_np_vps(s))
        c_score = coh_map[i]
        pos = get_sen_pos(i, t)
    
        rank_obj = {
            'tf_idf': tf_idf, 
            'sen_length': sen_length, 
            'proper_nouns': proper_nouns,
            'np_vps': np_vps,
            'c_score': c_score,
            'sen_pos': pos,
            'index': i
        }
        
        result_array.append(rank_obj)
        
        
    result = fuzzy_model(result_array)

    wr = open(os.path.join(curpath, '/tmp/score.json'), "w")
    for i, r in enumerate(result):
        r["sentence"] = sens[i]
        wr.write(json.dumps(r) + "\n")
    return result, sens

In [28]:
def fitness (individual, data):
    fitness = 0
    for (selected, (feature_label, score)) in zip(individual, data):
        if selected:
            fitness += score            
    return fitness

# def fitness (individual, data):
#     fitness = 0
#     if individual.count(1) == 3:
#         for (selected, (feature_label, score)) in zip(individual, data):
#             if selected:
#                 fitness += score    
#     return fitness

In [29]:
def ga_model(res_array):
    min_max_map = {}    
    result = [] 
#     senetnce_scores = []
        
    for k in ['tf_idf', 'sen_length', 'proper_nouns', 'np_vps', 'c_score', 'sen_pos']:
        min_max_map.setdefault(k, {})['max'] = max([x[k] for x in res_array])
        min_max_map[k]['min'] = min([x[k] for x in res_array])
    
        
    for r in res_array:
        
        tfIdf = get_val(r['tf_idf'], min_max_map['tf_idf']['min'], min_max_map['tf_idf']['max'])
        senLength = get_val(r['sen_length'], min_max_map['sen_length']['min'], min_max_map['sen_length']['max'])
        properNouns = get_val(r['proper_nouns'], min_max_map['proper_nouns']['min'], min_max_map['proper_nouns']['max']) 
        npVps = get_val(r['np_vps'], min_max_map['np_vps']['min'], min_max_map['np_vps']['max'])
        cScore = get_val(r['c_score'], min_max_map['c_score']['min'], min_max_map['c_score']['max']) 
        senPos = ((r['sen_pos'] - min_max_map['sen_pos']['min'])/(min_max_map['sen_pos']['max'] - min_max_map['sen_pos']['min']))
        
        data = [('tfidf', tfIdf), ('sen_length', senLength), ('proper_nouns', properNouns), ('np_vps', npVps), ('c_score', cScore), ('sen_pos', senPos)]
        
        ga = pyeasyga.GeneticAlgorithm(
            data,
            population_size = 100,
            generations = 100,
            crossover_probability = 0.8,
            mutation_probability = 0.2,
            elitism = True,
            maximise_fitness = True
        )
        
        ga.fitness_function = fitness
        
        ga.run()
        
        res = ga.best_individual()
        
        totalScore = res[0] * ts_float32(return_postive_value(tfIdf)) + res[0] * ts_float32(return_postive_value(senLength)) + res[0] * ts_float32(return_postive_value(properNouns)) + res[0] * ts_float32(return_postive_value(npVps)) + res[0] * ts_float32(return_postive_value(cScore)) + res[0] * ts_float32(return_postive_value(senPos))
        
#         totalScore = totalScore * res[0] 
        
        obj = {
            'tf_idf': res[0] * ts_float32(return_postive_value(tfIdf)),
            'sen_length': res[0] * ts_float32(return_postive_value(senLength)),
            'proper_nouns': res[0] * ts_float32(return_postive_value(properNouns)),
            'np_vps': res[0] * ts_float32(return_postive_value(npVps)),
            'c_score': res[0] * ts_float32(return_postive_value(cScore)),
            'sen_pos': res[0] * ts_float32(return_postive_value(senPos)),
#             'total_score': res[0],
            'total_score': totalScore,
            'index': r['index']
        }
#         senetnce_scores.append(res[0])
        result.append(obj)
        
#     for val in result:
#         val['total_score'] = (val['total_score'] - min(senetnce_scores)) / (max(senetnce_scores) - min(senetnce_scores))
    
    return result

In [30]:
def ga_ranking_model(text, D, df_map, model):
    curpath = os.path.abspath(os.curdir)
    sens = get_sentences(text)
    t = len(sens)
    coh_map = get_sen_cohesiveness(sens, model)
    result_array = []
    
    
    for i, s in enumerate(sens):
        # print(s)
        tf_idf = calculate_tf_idf_sum(s, D, df_map)
        sen_length = get_sen_length(s)
        proper_nouns = get_proper_nouns(s)
        np_vps = len(get_np_vps(s))
        c_score = coh_map[i]
        pos = get_sen_pos(i, t)
    
        rank_obj = {
            'tf_idf': tf_idf, 
            'sen_length': sen_length, 
            'proper_nouns': proper_nouns,
            'np_vps': np_vps,
            'c_score': c_score,
            'sen_pos': pos,
            'index': i
        }
        
        result_array.append(rank_obj)
        
        
    result = ga_model(result_array)

    wr = open(os.path.join(curpath, '/tmp/score.json'), "w")
    for i, r in enumerate(result):
        r["sentence"] = sens[i]
        wr.write(json.dumps(r) + "\n")
    return result, sens

In [31]:
def pso_model(res_array):
    min_max_map = {}    
    result = [] 
#     senetnce_scores = []
        
    for k in ['tf_idf', 'sen_length', 'proper_nouns', 'np_vps', 'c_score', 'sen_pos']:
        min_max_map.setdefault(k, {})['max'] = max([x[k] for x in res_array])
        min_max_map[k]['min'] = min([x[k] for x in res_array])
    
        
    for r in res_array:
        
        tfIdf = get_val(r['tf_idf'], min_max_map['tf_idf']['min'], min_max_map['tf_idf']['max'])
        senLength = get_val(r['sen_length'], min_max_map['sen_length']['min'], min_max_map['sen_length']['max'])
        properNouns = get_val(r['proper_nouns'], min_max_map['proper_nouns']['min'], min_max_map['proper_nouns']['max']) 
        npVps = get_val(r['np_vps'], min_max_map['np_vps']['min'], min_max_map['np_vps']['max'])
        cScore = get_val(r['c_score'], min_max_map['c_score']['min'], min_max_map['c_score']['max']) 
        senPos = ((r['sen_pos'] - min_max_map['sen_pos']['min'])/(min_max_map['sen_pos']['max'] - min_max_map['sen_pos']['min']))
        
#         options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
        options = {
            'c1': tfIdf, 
            'c2': senLength,
            'c3': properNouns,
            'c4': npVps,
            'c5': cScore,
            'c6': senPos,
            'w': 0.75
        }
        
        data  = [tfIdf, senLength, properNouns, npVps, cScore, senPos]
        
        
        optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=6, options=options)
        
        cost, pos = optimizer.optimize(fx.sphere, iters=100)
    
#         res = cost
        
        totalScore = cost * ts_float32(return_postive_value(tfIdf)) + cost * ts_float32(return_postive_value(senLength)) + cost * ts_float32(return_postive_value(properNouns)) + cost * ts_float32(return_postive_value(npVps)) + cost * ts_float32(return_postive_value(cScore)) + cost * ts_float32(return_postive_value(senPos))
        
#         totalScore = totalScore * res[0] 
        
        obj = {
            'tf_idf': cost * ts_float32(return_postive_value(tfIdf)),
            'sen_length': cost * ts_float32(return_postive_value(senLength)),
            'proper_nouns': cost * ts_float32(return_postive_value(properNouns)),
            'np_vps': cost * ts_float32(return_postive_value(npVps)),
            'c_score': cost * ts_float32(return_postive_value(cScore)),
            'sen_pos': cost * ts_float32(return_postive_value(senPos)),
#             'total_score': cost,
            'total_score': totalScore,
            'index': r['index']
        }
#         senetnce_scores.append(res[0])
        result.append(obj)
        
#     for val in result:
#         val['total_score'] = (val['total_score'] - min(senetnce_scores)) / (max(senetnce_scores) - min(senetnce_scores))
    
    return result

In [32]:
def pso_ranking_model(text, D, df_map, model):
    curpath = os.path.abspath(os.curdir)
    sens = get_sentences(text)
    t = len(sens)
    coh_map = get_sen_cohesiveness(sens, model)
    result_array = []
    
    
    for i, s in enumerate(sens):
        # print(s)
        tf_idf = calculate_tf_idf_sum(s, D, df_map)
        sen_length = get_sen_length(s)
        proper_nouns = get_proper_nouns(s)
        np_vps = len(get_np_vps(s))
        c_score = coh_map[i]
        pos = get_sen_pos(i, t)
    
        rank_obj = {
            'tf_idf': tf_idf, 
            'sen_length': sen_length, 
            'proper_nouns': proper_nouns,
            'np_vps': np_vps,
            'c_score': c_score,
            'sen_pos': pos,
            'index': i
        }
        
        result_array.append(rank_obj)
        
        
    result = pso_model(result_array)

    wr = open(os.path.join(curpath, '/tmp/score.json'), "w")
    for i, r in enumerate(result):
        r["sentence"] = sens[i]
        wr.write(json.dumps(r) + "\n")
    return result, sens

In [33]:
def rank__V2(res_array, senetnces):
    min_max_map = {}    
    result = [] 
    sents_ranks = []
#     senetnce_scores = []
        
    for k in ['tf_idf', 'sen_length', 'proper_nouns', 'np_vps', 'c_score', 'sen_pos']:
        min_max_map.setdefault(k, {})['max'] = max([x[k] for x in res_array])
        min_max_map[k]['min'] = min([x[k] for x in res_array])
        
    for r in res_array:
        
        tfIdf = get_val(r['tf_idf'], min_max_map['tf_idf']['min'], min_max_map['tf_idf']['max'])
        senLength = get_val(r['sen_length'], min_max_map['sen_length']['min'], min_max_map['sen_length']['max'])
        properNouns = get_val(r['proper_nouns'], min_max_map['proper_nouns']['min'], min_max_map['proper_nouns']['max']) 
        npVps = get_val(r['np_vps'], min_max_map['np_vps']['min'], min_max_map['np_vps']['max'])
        cScore = get_val(r['c_score'], min_max_map['c_score']['min'], min_max_map['c_score']['max']) 
#         senPos = ((r['sen_pos'] - min_max_map['sen_pos']['min'])/(min_max_map['sen_pos']['max'] - min_max_map['sen_pos']['min']))
        senPos = r['sen_pos']
    
        sent_text = senetnces[r['index']]
        
        res = keywords_extraction(sent_text, top_p = 1, directed=False)[0][:len(sent_text.split())]
        
        resValues = res.values()
        
        resTotal = 0
        
        for val in resValues:
            resTotal += val
        
        totalScore = ts_float32(return_postive_value(tfIdf)) + ts_float32(return_postive_value(senLength)) + ts_float32(return_postive_value(properNouns)) + ts_float32(return_postive_value(npVps)) + ts_float32(return_postive_value(cScore)) + ts_float32(return_postive_value(senPos))
        
#         totalScore = totalScore * resTotal
        
        obj = {
            'tf_idf': ts_float32(return_postive_value(tfIdf)),
            'sen_length': ts_float32(return_postive_value(senLength)),
            'proper_nouns': ts_float32(return_postive_value(properNouns)),
            'np_vps': ts_float32(return_postive_value(npVps)),
            'c_score': ts_float32(return_postive_value(cScore)),
            'sen_pos': ts_float32(return_postive_value(senPos)),
#             'total_score': res[0],
            'total_score': totalScore,
            'index': r['index']
        }
#         senetnce_scores.append(res[0])
        result.append(obj)
        
#     for val in result:
#         val['total_score'] = (val['total_score'] - min(senetnce_scores)) / (max(senetnce_scores) - min(senetnce_scores))
    
    return result

In [34]:
def ranking_model_v2(text, D, df_map, model):
    curpath = os.path.abspath(os.curdir)
    sens = get_sentences(text)
    t = len(sens)
    coh_map = get_sen_cohesiveness(sens, model)
    result_array = []
    
    
    for i, s in enumerate(sens):
        # print(s)
        tf_idf = calculate_tf_idf_sum(s, D, df_map)
        sen_length = get_sen_length(s)
        proper_nouns = get_proper_nouns(s)
        np_vps = len(get_np_vps(s))
        c_score = coh_map[i]
        pos = get_sen_pos(i, t)
    
        rank_obj = {
            'tf_idf': tf_idf, 
            'sen_length': sen_length, 
            'proper_nouns': proper_nouns,
            'np_vps': np_vps,
            'c_score': c_score,
            'sen_pos': pos,
            'index': i
        }
        
        result_array.append(rank_obj)
        
        
    result = rank__V2(result_array, sens)

    wr = open(os.path.join(curpath, '/tmp/score.json'), "w")
    for i, r in enumerate(result):
        r["sentence"] = sens[i]
        wr.write(json.dumps(r) + "\n")
    return result, sens

In [35]:
def get_sen_cluster_map(sens, model, n_clusters):
    X = get_sen_vec_list(sens, model)
#     kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
#     agglomerativeClustering = AgglomerativeClustering(n_clusters=None, compute_full_tree=True, distance_threshold=0.75).fit(X)
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(np.transpose(X), n_clusters, n_clusters, error=0.005, maxiter=1000, init=None)
#     labels = agglomerativeClustering.labels_
    labels = np.argmax(u, axis=0)
#     labels = kmeans.labels_
    return labels

In [36]:
def get_summary_indices(cluster_score_map, sens, sum_words):
    sorted_cl_map = {}
    for key, score_map in cluster_score_map.items():
        sorted_by_value = sorted(score_map.items(), key=lambda kv: kv[1], reverse=True)
        sorted_cl_map[key] = sorted_by_value

    # Get final sentence indices.
    final_sumamry_indices = []
    x = 0
    i = 0
    print(' summary words ', sum_words)
    while (x < sum_words):
        for key, score_map in sorted_cl_map.items():

            # The below if will ensure that  current cluster has ith senetence, it can not be always the case.
            if (len(score_map) > i):

                # This condition is for breaking during loop if num words are conaition is stastified.
                if x >= sum_words:
                    break

                index = score_map[i][0]
                token_len = len(nl.word_tokenize(sens[index]))
                if token_len > 15:
                    final_sumamry_indices.append(index)
                    # TODO : don't consider those sentences whose length is less than certain threshhold.
                    x += token_len

        i += 1
    return final_sumamry_indices

In [37]:
def get_summary_sentences(summary_indices, sens):
    return [sens[x] for x in summary_indices]

In [38]:
def calculate_df():
    stop_words = stopwords.words('english') + list(punctuation)

    df_map = {}
    print("going to read corpus for calculating document frequency")
    for file_id in reuters.fileids():
        # TO Get unique occurrence of word
        words = set(tokenize(reuters.raw(file_id), stop_words))
        for w in words:
            df_map.setdefault(w, df_map.get(w, 0) + 1)
    return df_map, len(reuters.fileids())

In [39]:
def get_summary(input_file_path, sum_percent, model_path):
    word_to_vec_model = load_model(model_path)
    df_map, D = calculate_df()

    with open(input_file_path, 'r') as my_file:
        data = my_file.read().replace('\n', '')
        data = data.replace(".", ". ")

#     result_array, sens = get_final_sen_ranks(data, D, df_map, word_to_vec_model)
#     result_array, sens = fuzzy_ranking_model(data, D, df_map, word_to_vec_model)
#     result_array, sens = ga_ranking_model(data, D, df_map, word_to_vec_model)
    result_array, sens = pso_ranking_model(data, D, df_map, word_to_vec_model)
#     result_array, sens = ranking_model_v2(data, D, df_map, word_to_vec_model)
    cluster_labels = get_sen_cluster_map(sens, word_to_vec_model, n_clusters=2)
    total_words = sum([len(nl.word_tokenize(sen_tokens)) for sen_tokens in sens])
    sum_words = math.ceil((sum_percent / 100) * total_words)
    final_cluster_score_map = {}
    for r in result_array:
        index = r['index']
        total_score = r['total_score']
        cluster = int(cluster_labels[index])
        final_cluster_score_map.setdefault(cluster, {})[index] = total_score
    # print(final_cluster_score_map)
    summary_indices = get_summary_indices(final_cluster_score_map, sens, sum_words)
    summary_indices.sort()
    # print(summary_indices)
    return get_summary_sentences(summary_indices, sens)

# Final Run

In [40]:
# summary_percent = 20
# # input_file = r"C:\Users\Mohsin\Desktop\Semantic-text-summarizer-master\data\sample_text.txt"
# input_file = r"C:\Users\Mohsin\Desktop\Desktop\test_doc.txt"
# model_path = r"C:\Users\Mohsin\Desktop\wrd2vec\word2vec-google-news-300.gz"
# summary_sens = get_summary(input_file, summary_percent, model_path)

In [41]:
# print("Summary:\n")
# for s in summary_sens:
#     print(s)

In [42]:
# print('Orignal Text:\n', open(input_file).read()

In [43]:
# files_paths = [
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_1\DOC_1.txt',
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_2\DOC_2.txt',
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_3\DOC_3.txt',
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_4\DOC_4.txt',
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_5\DOC_5.txt',
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_6\DOC_6.txt',
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_7\DOC_7.txt',
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_8\DOC_8.txt',
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_9\DOC_9.txt',
#     r'E:\Thesis\Documments_+_Ref_Summries\DOC_10\DOC_10.txt'
# ]

files_paths = [
    r'E:\Thesis\Dataset_2\DOC_1\DOC_1.txt',
    r'E:\Thesis\Dataset_2\DOC_2\DOC_2.txt',
    r'E:\Thesis\Dataset_2\DOC_3\DOC_3.txt',
    r'E:\Thesis\Dataset_2\DOC_4\DOC_4.txt',
    r'E:\Thesis\Dataset_2\DOC_5\DOC_5.txt',
    r'E:\Thesis\Dataset_2\DOC_6\DOC_6.txt',
    r'E:\Thesis\Dataset_2\DOC_7\DOC_7.txt',
    r'E:\Thesis\Dataset_2\DOC_8\DOC_8.txt',
    r'E:\Thesis\Dataset_2\DOC_9\DOC_9.txt',
    r'E:\Thesis\Dataset_2\DOC_10\DOC_10.txt',
    r'E:\Thesis\Dataset_2\DOC_11\DOC_11.txt',
    r'E:\Thesis\Dataset_2\DOC_12\DOC_12.txt',
    r'E:\Thesis\Dataset_2\DOC_13\DOC_13.txt',
    r'E:\Thesis\Dataset_2\DOC_14\DOC_14.txt',
    r'E:\Thesis\Dataset_2\DOC_15\DOC_15.txt',
    r'E:\Thesis\Dataset_2\DOC_16\DOC_16.txt',
    r'E:\Thesis\Dataset_2\DOC_17\DOC_17.txt',
    r'E:\Thesis\Dataset_2\DOC_18\DOC_18.txt',
    r'E:\Thesis\Dataset_2\DOC_19\DOC_19.txt',
    r'E:\Thesis\Dataset_2\DOC_20\DOC_20.txt'
]

In [44]:
summary_percent = 25
model_path = r"C:\Users\Mohsin\Desktop\wrd2vec\word2vec-google-news-300.gz"
summries = []
count = 1
for file in files_paths:
    summary_sens = get_summary(file, summary_percent, model_path)
    
    obj = {
            'DOC_' + str(count) : summary_sens,
    }
    
    summries.append(obj)
    
    count += 1

    
wr = open(os.path.join(os.path.abspath(os.curdir), '/tmp/result_poposed_model.json'), "w")
for summ in summries:
    wr.write(json.dumps(summ) + "\n\n")
    
wr.close()

2021-04-26 22:17:32,864 - gensim.models.utils_any2vec - INFO - loading projection weights from C:\Users\Mohsin\Desktop\wrd2vec\word2vec-google-news-300.gz
2021-04-26 22:18:04,929 - gensim.models.utils_any2vec - INFO - loaded (3000000, 300) matrix from C:\Users\Mohsin\Desktop\wrd2vec\word2vec-google-news-300.gz


going to read corpus for calculating document frequency


2021-04-26 22:18:17,594 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.7894736842105263, 'c2': 0.7419354838709677, 'c3': 0.45454545454545453, 'c4': 1.0, 'c5': 0.900523, 'c6': 1.0, 'w': 0.75}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████|100/100, best_cost=6.31e-11
2021-04-26 22:18:17,713 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 6.306013314071113e-11, best pos: [ 2.02920872e-06  1.56552220e-06 -4.87704900e-06 -1.31266800e-06
  5.33630362e-06 -1.58327027e-06]
2021-04-26 22:18:17,723 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.31578947368421045, 'c2': 0.5483870967741935, 'c3': 0.09090909090909091, 'c4': 0.8571428571428571, 'c5': 0.33995503, 'c6': 0.9166666666666667, 'w': 0.75}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=9.02e-9
2021-04-26 22:18:17,812 - pyswarms.single.global_best - I

 summary words  104


2021-04-26 22:18:51,427 - gensim.models.utils_any2vec - INFO - loaded (3000000, 300) matrix from C:\Users\Mohsin\Desktop\wrd2vec\word2vec-google-news-300.gz


...

 summary words  34


2021-04-26 22:32:58,266 - gensim.models.utils_any2vec - INFO - loaded (3000000, 300) matrix from C:\Users\Mohsin\Desktop\wrd2vec\word2vec-google-news-300.gz


going to read corpus for calculating document frequency


2021-04-26 22:33:10,973 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.6206896551724137, 'c2': 0.5576923076923077, 'c3': 1.0, 'c4': 0.6153846153846154, 'c5': 0.75377584, 'c6': 1.0, 'w': 0.75}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████|100/100, best_cost=1.76e-10
2021-04-26 22:33:11,091 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.7554013879776753e-10, best pos: [-2.89676286e-06  3.89642421e-06  5.69639065e-07  3.82850016e-06
 -1.13860762e-05 -2.70963980e-06]
2021-04-26 22:33:11,097 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.3448275862068965, 'c2': 0.25, 'c3': 1.0, 'c4': 0.23076923076923078, 'c5': 0.73926455, 'c6': 0.8750000000000001, 'w': 0.75}
pyswarms.single.global_best: 100%|███████████████████████████████████████████████████████████|100/100, best_cost=0.0038
2021-04-26 22:33:11,176 - pyswarms.single.global_best - INFO - Optimization finished 

 summary words  71
